In [ ]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import pandas as pd
import numpy as np
import re 
import ast
import itertools

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split

from bs4 import BeautifulSoup

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
n = 100

'Month', 'Weekday', 'Year', 'Day', 'Hour', 'title_bit_count', 'img_count', 'link_count', 'title_word_count', 'content_len', 'content_word_count'

-----------------------------------------------------------
括號內是 acc > 0.58 的權重
'Month', 'Weekday', 'Year', 'Hour' -> 100% (100%)

'Day' -> 85% (72%)

'img_count' -> 89% (65%)

'channel' -> 81%

'content_len' -> 49% (48%) -> 59%

'author' -> 56%

'title_bit_count' -> 36% (48%) -> 53%

'link_count' -> 40% (46%) -> 37%

'title_word_count' -> 36% (49%) -> 37% (drop?)

'content_word_count' -> 38% (47%) (drop)

'categories_count' -> 16% (15%) (drop)

'Minutes' -> 跟 categories_count 差不多 (drop)

**author and channel is important**
channel 權重大於 author

In [ ]:
df = pd.read_csv('../datasets_processed/train_processed_2.csv')
print(df.columns.values)

In [ ]:
# extra features : title, content, categories
X_train = df[['Month', 'Weekday', 'Year', 'Day', 'Hour', 'img_count', 'content_len', 'channel', 'author', 'categories']]
y_train = df.iloc[:]['Popularity'].values
y_train[y_train==-1] = 0

In [ ]:
X_train['title'] = X_train['title'].apply(ast.literal_eval)
X_train['categories'] = X_train['categories'].apply(ast.literal_eval)
X_train['channel'] = X_train['channel'].apply(ast.literal_eval)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
stop = stopwords.words('english')
stemmer = PorterStemmer()

def tokenizer_author(text):
    if type(text) == np.ndarray:
        text = text[0]
    authors = re.split(',', text)
    for idx, author in enumerate(authors):
        authors[idx] = re.sub(' ', '', author)
    return authors
    
def tokenizer_channel(text):
    if type(text) == np.ndarray:
        text = text[0]
    channels = re.split(',', text)
    for idx, channel in enumerate(channels):
        channels[idx] = re.sub(' ', '', channel)
    return channels

def tokenizer_title(list):
    # word-stemming
    stemmed_title = [stemmer.stem(word) for word in list]
    # remove stopwords
    cleaned_title = [word for word in stemmed_title if word not in stop]
    return cleaned_title


def tokenizer_categories(list):   
    # word-stemming
    stemmed_categories = [stemmer.stem(word) for word in list]
    # remove stopwords
    cleaned_categories = [word for word in stemmed_categories if word not in stop]
    return cleaned_categories

In [ ]:
# X_train['title'] = X_train['title'].apply(tokenizer_title)
# X_train['title'].to_csv('../datasets_processed/title.csv', index=False)

In [ ]:
# preprocess the author
vectorizer = CountVectorizer(tokenizer=tokenizer_author, lowercase=False)
vectorized_data = vectorizer.fit_transform(X_train['author'])
vectorized_author = pd.DataFrame(vectorized_data.toarray(), columns=vectorizer.get_feature_names_out())

# preprocess the channel
vectorizer = CountVectorizer(tokenizer=tokenizer_channel, lowercase=False)
vectorized_data = vectorizer.fit_transform(X_train['channel'])
vectorized_channel = pd.DataFrame(vectorized_data.toarray(), columns=vectorizer.get_feature_names_out())

# preprocess the title
# vectorizer = CountVectorizer(tokenizer=tokenizer_title, lowercase=False)
# vectorized_data = vectorizer.fit_transform(X_train['title'])
# vectorized_title = pd.DataFrame(vectorized_data.toarray(), columns=vectorizer.get_feature_names_out())

# preprocess the categories
vectorizer = CountVectorizer(tokenizer=tokenizer_categories, lowercase=False)
vectorized_data = vectorizer.fit_transform(X_train['categories'])
vectorized_categories = pd.DataFrame(vectorized_data.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
# print(vectorized_categories.shape)

In [ ]:
print(X_train.columns.values)

In [ ]:
# Doing standardization for numerical features
scaler = StandardScaler()
X_train[['Month', 'Weekday', 'Year', 'Day', 'Hour', 'img_count', 'content_len', ]] = scaler.fit_transform(X_train[['Month', 'Weekday', 'Year', 'Day', 'Hour', 'img_count', 'content_len']])

In [ ]:
X_train = X_train.drop(['author'], axis=1)
X_train = X_train.drop(['channel'], axis=1)
X_train = X_train.drop(['categories'], axis=1)
# X_train = pd.concat([X_train, vectorized_author], axis=1)
# X_train = pd.concat([X_train, vectorized_channel], axis=1)
X_train = pd.concat([X_train, vectorized_categories], axis=1)


In [ ]:
lbgm = LGBMClassifier(n_estimators=n, max_depth=10, learning_rate=0.1, random_state=0)
scores = cross_val_score(estimator=lbgm, X=X_train.values, y=y_train, cv=5, scoring='roc_auc')
print('%.3f (+/-%.3f)' % (scores.mean(), scores.std()))

In [ ]:
com = []
acc = []
attributes = X_train.columns.values
for r in range(1, len(attributes) + 1):
    for combination in itertools.combinations(attributes, r):
        # if combination.__contains__('Month') and combination.__contains__('Weekday') and combination.__contains__('Year') and \
        #     combination.__contains__('Day') and combination.__contains__('Hour') and combination.__contains__('img_count') and \
        #     combination.__contains__('channel'):
            tmp = X_train[list(combination)]
            if combination.__contains__('author'):
                tmp = tmp.drop(['author'], axis=1)
                tmp = pd.concat([tmp, vectorized_author], axis=1)
            if combination.__contains__('categories'):
            # if combination.__contains__('title'):
            #     tmp = tmp.drop(['title'], axis=1)
            #     tmp = pd.concat([tmp, vectorized_title], axis=1)
            if combination.__contains__('channel'):
                tmp = tmp.drop(['channel'], axis=1)
                tmp = pd.concat([tmp, vectorized_channel], axis=1)
            # if combination.__contains__('content'):
            lbgm = LGBMClassifier(n_estimators=n, max_depth=10, learning_rate=0.1, random_state=1, num_leaves=100)
            scores = cross_val_score(estimator=lbgm, X=tmp.values, y=y_train, cv=5, scoring='roc_auc')
            com.append(combination)
            acc.append([scores.mean(), scores.std()])


tmp = pd.DataFrame({'combination': com, 'accuracy': acc})

In [ ]:
# output validation result
sorted_tmp = tmp.sort_values(by=['accuracy'], ascending=False)
sorted_tmp.to_csv('../training_output/lgbm_2_acc.csv', index=False)

In [ ]:
rf = RandomForestClassifier(n_estimators=10, random_state=15)
scores = cross_val_score(estimator=rf, X=tmp.values, y=y_train, cv=5, scoring='roc_auc')
print('%.3f (+/-%.3f)' % (scores.mean(), scores.std()))

In [ ]:
com = []
acc = []
attributes = X_train.columns.values
for r in range(1, len(attributes) + 1):
    for combination in itertools.combinations(attributes, r):
        # if combination.__contains__('Month') and combination.__contains__('Weekday') and combination.__contains__('Year') and \
        #     combination.__contains__('Day') and combination.__contains__('Hour') and combination.__contains__('img_count') and \
        #     combination.__contains__('channel'):
            tmp = X_train[list(combination)]
            if combination.__contains__('author'):
                tmp = tmp.drop(['author'], axis=1)
                tmp = pd.concat([tmp, vectorized_author], axis=1)
            # if combination.__contains__('categories'):
            # if combination.__contains__('title'):
            #     tmp = tmp.drop(['title'], axis=1)
            #     tmp = pd.concat([tmp, vectorized_title], axis=1)
            if combination.__contains__('channel'):
                tmp = tmp.drop(['channel'], axis=1)
                tmp = pd.concat([tmp, vectorized_channel], axis=1)
            rf = RandomForestClassifier(n_estimators=10, random_state=15)
            scores = cross_val_score(estimator=rf, X=tmp.values, y=y_train, cv=5, scoring='roc_auc')
            com.append(combination)
            acc.append([scores.mean(), scores.std()])


tmp = pd.DataFrame({'combination': com, 'accuracy': acc})

In [ ]:
tmp = pd.DataFrame({'combination': com, 'accuracy': acc})

In [ ]:
# output validation result
sorted_tmp = tmp.sort_values(by=['accuracy'], ascending=False)
sorted_tmp.to_csv('../training_output/random_forest_acc.csv', index=False)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(estimator=knn, X=X_train.values, y=y_train, cv=5, scoring='roc_auc')
print('%.3f (+/-%.3f)' % (scores.mean(), scores.std()))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# X_train = pd.concat([X_train, vectorized_author], axis=1)
# X_train = pd.concat([X_train, vectorized_channel], axis=1)
# X_train = pd.concat([X_train, vectorized_categories], axis=1)
X_train = df[['Month', 'Weekday', 'Year', 'Day', 'Hour', 'img_count', 'categories', 'author', 'channel']]
com = []
acc = []
attributes = X_train.columns.values
for r in range(1, len(attributes) + 1):
    for combination in itertools.combinations(attributes, r):
      if combination.__contains__('Month') and combination.__contains__('Weekday') and combination.__contains__('Year') :
        tmp = X_train[list(combination)]
        if combination.__contains__('author'):
          tmp = tmp.drop(['author'], axis=1)
          tmp = pd.concat([tmp, vectorized_author], axis=1)
        if combination.__contains__('categories'):
          tmp = tmp.drop(['categories'], axis=1)
          tmp = pd.concat([tmp, vectorized_categories], axis=1)
        if combination.__contains__('channel'):
          tmp = tmp.drop(['channel'], axis=1)
          tmp = pd.concat([tmp, vectorized_channel], axis=1)
        knn = KNeighborsClassifier(n_neighbors=400)
        scores = cross_val_score(estimator=knn, X=tmp.values, y=y_train, cv=5, scoring='roc_auc')
        com.append(combination)
        acc.append([scores.mean(), scores.std()])
        print('%.3f (+/-%.3f)' % (scores.mean(), scores.std()))
      

In [ ]:
# output validation result
tmp = pd.DataFrame({'combination': com, 'accuracy': acc})
sorted_tmp = tmp.sort_values(by=['accuracy'], ascending=False)
sorted_tmp.to_csv('../training_output/KNN_600_acc.csv', index=False)